In [1]:
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torch.nn.functional as F

from torch import nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import os
from einops import rearrange, repeat
import einops
from glob import glob
from math import log
import math
import kagglehub
from torchvision.datasets import CIFAR10
from torchvision.utils import save_image

os.environ['KAGGLEHUB_CACHE'] = '/pscratch/sd/t/tbalasoo' #Change this line to your location

/global/cfs/cdirs/m4727/mamba-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Install dependencies as needed:
# CelebA 64x64
import kagglehub

# Download latest version
path = kagglehub.dataset_download("therealcyberlord/50k-celeba-dataset-64x64")

print("Path to dataset files:", path)

#Path to dataset files: /home/idies/.cache/kagglehub/datasets/therealcyberlord/50k-celeba-dataset-64x64/versions/1


100%|██████████| 78.1M/78.1M [00:02<00:00, 36.8MB/s]

Extracting files...


Path to dataset files: /home/idies/.cache/kagglehub/datasets/therealcyberlord/50k-celeba-dataset-64x64/versions/1


In [5]:
#CelebA 178x178
import kagglehub
path = kagglehub.dataset_download("jessicali9530/celeba-dataset")

print("Path to dataset files:", path)
#Path to dataset files: /home/idies/.cache/kagglehub/datasets/jessicali9530/celeba-dataset/versions/2/img_align_celeba/img_align_celeba

100%|██████████| 1.33G/1.33G [00:06<00:00, 204MB/s] 

Extracting files...


Path to dataset files: /pscratch/sd/t/tbalasoo/datasets/jessicali9530/celeba-dataset/versions/2


In [2]:
#CelebA HQ 1024x1024
import shutil
from tqdm import tqdm
import os
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lamsimon/celebahq")

print("Path to dataset files:", path)

source_root = '/home/idies/.cache/kagglehub/datasets/lamsimon/celebahq/versions/1/celeba_hq'  # e.g., 'data/' with subfolders
dest_dir = '/home/idies/.cache/kagglehub/datasets/lamsimon/celebahq/versions/1/celeba_hq/data'
allowed_exts = {'.jpg', '.jpeg', '.png', '.bmp'}  # accepted image formats

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

all_files = []

# Walk through all subdirectories of source_root
for root, dirs, files in os.walk(source_root):
    for fname in files:
        ext = os.path.splitext(fname)[-1].lower()
        if ext in allowed_exts:
            full_path = os.path.join(root, fname)
            all_files.append(full_path)

# Remove duplicates (in case of symlinks or same name across dirs)
all_files = list(set(all_files))

print(f"Found {len(all_files)} unique image files.")

# Copy them to dest_dir
for i, src_path in enumerate(tqdm(all_files)):
    ext = os.path.splitext(src_path)[-1].lower()
    dst_path = os.path.join(dest_dir, f'img_{i:05}{ext}')
    shutil.copy2(src_path, dst_path)
    
#Path to dataset files: /home/idies/.cache/kagglehub/datasets/lamsimon/celebahq/versions/1

100%|██████████| 2.55G/2.55G [00:45<00:00, 59.6MB/s]

Extracting files...


Path to dataset files: /home/idies/.cache/kagglehub/datasets/lamsimon/celebahq/versions/1
Found 30000 unique image files.


100%|██████████| 30000/30000 [00:06<00:00, 4722.93it/s]


In [2]:
#CIFAR-10

#output_dir = '/home/idies/workspace/Temporary/tbalasoor/scratch/cifar10'
#output_dir = '/home/idies/.cache/cifar-10'
output_dir = '/pscratch/sd/t/tbalasoo/datasets/cifar-10'
os.makedirs(output_dir, exist_ok=True)

# Transformation
to_tensor = transforms.ToTensor()

# Download CIFAR-10
train_set = CIFAR10(root=output_dir, train=True, download=True)
test_set = CIFAR10(root=output_dir, train=False, download=True)

# Save all images from train (0 to 49999)
for i, (img, _) in tqdm(enumerate(train_set), total=len(train_set), desc="Saving train"):
    save_image(to_tensor(img), os.path.join(output_dir, f"{i:06d}.jpg"))

# Save all images from test (50000 to 59999)
for i, (img, _) in tqdm(enumerate(test_set), total=len(test_set), desc="Saving test"):
    idx = i + len(train_set)
    save_image(to_tensor(img), os.path.join(output_dir, f"{idx:06d}.jpg"))

100%|██████████| 170M/170M [00:04<00:00, 34.2MB/s] 


Extracting /pscratch/sd/t/tbalasoo/datasets/cifar-10/cifar-10-python.tar.gz to /pscratch/sd/t/tbalasoo/datasets/cifar-10
Files already downloaded and verified


Saving test: 100%|██████████| 10000/10000 [00:27<00:00, 368.52it/s]


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arnaud58/flickrfaceshq-dataset-ffhq")

print("Path to dataset files:", path)
# /home/idies/.cache/kagglehub/datasets/arnaud58/flickrfaceshq-dataset-ffhq/versions/1

100%|██████████| 19.5G/19.5G [05:13<00:00, 66.6MB/s]  

Extracting files...


Path to dataset files: /home/idies/.cache/kagglehub/datasets/arnaud58/flickrfaceshq-dataset-ffhq/versions/1


In [7]:
from pathlib import Path
from tqdm import tqdm
import shutil

# Set your source and destination directories
image_dir = Path("/home/idies/.cache/kagglehub/datasets/arnaud58/flickrfaceshq-dataset-ffhq/versions/1")
new_image_dir = Path("/home/idies/workspace/Temporary/tbalasoor/scratch/ffhq")

# Ensure the destination directory exists
new_image_dir.mkdir(parents=True, exist_ok=True)

# Get all .png files
png_files = sorted(image_dir.glob("*.png"))

# Move and rename
for idx, file in enumerate(tqdm(png_files)):
    new_name = new_image_dir / f"{idx:06d}.png"
    shutil.move(str(file), str(new_name))

print(f"Moved and renamed {len(png_files)} PNG files to {new_image_dir}.")




 45%|████▌     | 23495/52001 [49:28<1:01:12,  7.76it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 52001/52001 [1:51:08<00:00,  7.80it/s]  

Moved and renamed 52001 PNG files to /home/idies/workspace/Temporary/tbalasoor/scratch/ffhq.
